### Restart and Run All Cells

In [1]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()
year = 2021
quarter = 3

In [2]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit percent'.split()

In [3]:
format_dict = {
                'q_amt':'{:,}','q_amt_c':'{:,}','q_amt_p':'{:,}','inc_profit':'{:,}',
                'yoy_gain':'{:,}','acc_gain':'{:,}',   
                'latest_amt':'{:,}','previous_amt':'{:,}','inc_amt':'{:,}',
                'q_eps':'{:.4f}','y_eps':'{:.4f}','aq_eps':'{:.4f}','ay_eps':'{:.4f}',
                'percent':'{:.2f}%','inc_pct':'{:.2f}%'
              }

In [4]:
sql = """
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s) 
OR (year = %s-1 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC
"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2021 AND quarter <= 3) 
OR (year = 2021-1 AND quarter >= 3+1)
ORDER BY year DESC, quarter DESC



In [5]:
dfc = pd.read_sql(sql, conlt)
dfc["Counter"] = 1
dfc_grp = dfc.groupby(["name"], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp["Counter"] == 4]
dfc_grp.head()

,name,year,quarter,q_amt,Counter
0,2S,8083,10,788297,4
1,AAV,8083,10,-6769190,4
2,ACE,8083,10,1392091,4
3,ADVANC,8083,10,27223149,4
4,AEONTS,8083,10,3826113,4


In [6]:
sql = """
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s-1) 
OR (year = %s-1 AND quarter >= %s) 
ORDER BY year DESC, quarter DESC"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2021 AND quarter <= 3-1) 
OR (year = 2021-1 AND quarter >= 3) 
ORDER BY year DESC, quarter DESC


In [7]:
dfp = pd.read_sql(sql, conlt)
dfp["Counter"] = 1
dfp_grp = dfp.groupby(["name"], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp["Counter"] == 4]
dfp_grp.head().style.format(format_dict)

,name,year,quarter,q_amt,Counter
0,2S,8082,10,"725,410",4
1,AAV,8082,10,"-6,507,746",4
2,ACE,8082,10,"1,391,075",4
3,ADVANC,8082,10,"27,361,758",4
4,AEONTS,8082,10,"4,020,604",4


In [8]:
dfp.name.unique().shape

(338,)

In [9]:
sql = """
SELECT *
FROM stocks
"""
stocks = pd.read_sql(sql, conlt)
stocks.shape

(338, 15)

In [10]:
sql = """
SELECT *
FROM tickers
"""
tickers = pd.read_sql(sql, conlt)
tickers.shape

(402, 9)

In [8]:
dfm = pd.merge(dfc_grp, dfp_grp, on="name", suffixes=(["_c", "_p"]), how="inner")
dfm["inc_profit"] = dfm["q_amt_c"] - dfm["q_amt_p"]
dfm["percent"] = dfm["inc_profit"] / abs(dfm["q_amt_p"]) * 100
dfm["year"] = year
dfm["quarter"] = "Q" + str(quarter)
df_percent = dfm[cols]
df_percent.head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,2S,2021,Q3,"788,297","725,410","62,887",8.67%
1,AAV,2021,Q3,"-6,769,190","-6,507,746","-261,444",-4.02%
2,ACE,2021,Q3,"1,392,091","1,391,075","1,016",0.07%
3,ADVANC,2021,Q3,"27,223,149","27,361,758","-138,609",-0.51%
4,AEONTS,2021,Q3,"3,826,113","4,020,604","-194,491",-4.84%


In [9]:
sql = """
DELETE FROM qt_profits 
WHERE year = %s AND quarter = 'Q%s'
"""
sql = sql % (year, quarter)
print(sql)

rp = conlt.execute(sql)
rp.rowcount


DELETE FROM qt_profits 
WHERE year = 2021 AND quarter = 'Q3'



319

In [10]:
sql = "SELECT name, id FROM tickers"
tickers = pd.read_sql(sql, conlt)
tickers.head()

,name,id
0,2S,727
1,A,1
2,AAV,732
3,ACE,698
4,ADVANC,6


In [11]:
df_ins = pd.merge(df_percent, tickers, on="name", how="inner")
df_ins.head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
0,2S,2021,Q3,"788,297","725,410","62,887",8.67%,727
1,AAV,2021,Q3,"-6,769,190","-6,507,746","-261,444",-4.02%,732
2,ACE,2021,Q3,"1,392,091","1,391,075","1,016",0.07%,698
3,ADVANC,2021,Q3,"27,223,149","27,361,758","-138,609",-0.51%,6
4,AEONTS,2021,Q3,"3,826,113","4,020,604","-194,491",-4.84%,7


In [12]:
rcds = df_ins.values.tolist()
len(rcds)

319

In [13]:
#for rcd in rcds:
#    print(rcd)

In [14]:
sql = """
INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, \
inc_amt, inc_pct, ticker_id) \
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""
print(sql)


INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?)



In [15]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [16]:
criteria_1 = df_ins.q_amt_c > 440_000
df_ins.loc[criteria_1, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,2S,2021,Q3,"788,297","725,410","62,887",8.67%
2,ACE,2021,Q3,"1,392,091","1,391,075","1,016",0.07%
3,ADVANC,2021,Q3,"27,223,149","27,361,758","-138,609",-0.51%
4,AEONTS,2021,Q3,"3,826,113","4,020,604","-194,491",-4.84%
5,AH,2021,Q3,"1,041,977","1,109,197","-67,220",-6.06%


In [17]:
criteria_2 = df_ins.q_amt_p > 400_000
df_ins.loc[criteria_2, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,2S,2021,Q3,"788,297","725,410","62,887",8.67%
2,ACE,2021,Q3,"1,392,091","1,391,075","1,016",0.07%
3,ADVANC,2021,Q3,"27,223,149","27,361,758","-138,609",-0.51%
4,AEONTS,2021,Q3,"3,826,113","4,020,604","-194,491",-4.84%
5,AH,2021,Q3,"1,041,977","1,109,197","-67,220",-6.06%


In [18]:
criteria_3 = df_ins.percent > 10.00
df_ins.loc[criteria_3, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
15,AMC,2021,Q3,"809,057","665,184","143,873",21.63%
24,ASP,2021,Q3,"907,856","818,663","89,193",10.89%
25,AWC,2021,Q3,"-597,662","-1,905,093","1,307,431",68.63%
28,BAM,2021,Q3,"2,321,436","2,043,406","278,030",13.61%
29,BANPU,2021,Q3,"5,897,490","1,875,967","4,021,523",214.37%


In [19]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
15,AMC,2021,Q3,"809,057","665,184","143,873",21.63%
24,ASP,2021,Q3,"907,856","818,663","89,193",10.89%
28,BAM,2021,Q3,"2,321,436","2,043,406","278,030",13.61%
29,BANPU,2021,Q3,"5,897,490","1,875,967","4,021,523",214.37%
31,BBL,2021,Q3,"22,586,660","19,694,952","2,891,708",14.68%


In [20]:
df_ins[df_ins_criteria].sort_values(by=["percent"], ascending=[False]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
29,BANPU,2021,Q3,"5,897,490","1,875,967","4,021,523",214.37%,47
308,VIH,2021,Q3,"1,260,420","462,452","797,968",172.55%,611
194,PSL,2021,Q3,"2,730,124","1,206,757","1,523,367",126.24%,734
32,BCH,2021,Q3,"4,644,042","2,161,170","2,482,872",114.89%,51
146,LANNA,2021,Q3,"1,312,902","639,500","673,402",105.30%,264


In [21]:
df_ins[df_ins_criteria].sort_values(by=["name"], ascending=[True]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
15,AMC,2021,Q3,"809,057","665,184","143,873",21.63%,22
24,ASP,2021,Q3,"907,856","818,663","89,193",10.89%,40
28,BAM,2021,Q3,"2,321,436","2,043,406","278,030",13.61%,709
29,BANPU,2021,Q3,"5,897,490","1,875,967","4,021,523",214.37%,47
31,BBL,2021,Q3,"22,586,660","19,694,952","2,891,708",14.68%,50


In [22]:
df_ins[df_ins_criteria].sort_values(by=["name"], ascending=[True]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
15,AMC,2021,Q3,"809,057","665,184","143,873",21.63%,22
24,ASP,2021,Q3,"907,856","818,663","89,193",10.89%,40
28,BAM,2021,Q3,"2,321,436","2,043,406","278,030",13.61%,709
29,BANPU,2021,Q3,"5,897,490","1,875,967","4,021,523",214.37%,47
31,BBL,2021,Q3,"22,586,660","19,694,952","2,891,708",14.68%,50


In [23]:
sql = """
SELECT name, COUNT(*) AS 'Nbr. of Group of 4 Quarters'
FROM qt_profits
GROUP BY name
HAVING COUNT(*) = 1
ORDER BY COUNT(*) DESC, name
"""
print(sql)

tmp = pd.read_sql(sql, conlt)
tmp


SELECT name, COUNT(*) AS 'Nbr. of Group of 4 Quarters'
FROM qt_profits
GROUP BY name
HAVING COUNT(*) = 1
ORDER BY COUNT(*) DESC, name



,name,Nbr. of Group of 4 Quarters
0,BCT,1
1,INSET,1
2,NRF,1
3,OR,1
4,UOBKH,1


In [24]:
name = 'OR'
sql = """
SELECT *
FROM epss
WHERE name = '%s'
ORDER BY year DESC, quarter DESC
"""
sql = sql % name
print(sql)

epss = pd.read_sql(sql, conlt)
epss


SELECT *
FROM epss
WHERE name = 'OR'
ORDER BY year DESC, quarter DESC



,id,name,year,quarter,q_amt,y_amt,aq_amt,ay_amt,q_eps,y_eps,aq_eps,ay_eps,ticker_id,publish_date
0,20753,OR,2021,3,1892478,3450043,9120618,5868493,0.16,0.38,0.79,0.65,715,2021-11-09
1,20266,OR,2021,2,3224942,520104,7228140,2418450,0.27,0.06,0.64,0.27,715,2021-08-10
2,19788,OR,2021,1,4003198,1898346,4003198,1898346,0.38,0.21,0.38,0.21,715,2021-05-11
3,19258,OR,2020,4,2922576,1948351,8791070,10895805,0.33,0.22,0.98,1.21,715,2021-02-17
4,19257,OR,2020,3,3450043,2593929,5868494,8947454,0.38,0.29,0.65,0.99,715,2021-02-17


In [25]:
sql = """
SELECT name, COUNT(*) AS 'Nbr. of Quarters'
FROM epss
GROUP BY name
HAVING COUNT(*) < 5
ORDER BY COUNT(*) DESC, name
"""
print(sql)

tmp = pd.read_sql(sql, conlt)
tmp


SELECT name, COUNT(*) AS 'Nbr. of Quarters'
FROM epss
GROUP BY name
HAVING COUNT(*) < 5
ORDER BY COUNT(*) DESC, name



,name,Nbr. of Quarters
0,ASW,3
1,SABUY,3
2,TIDLOR,3
3,TQR,3
